## Accessing ASTER L1T data with the Planetary Computer STAC API

In [1]:
from pystac_client import API
import planetary_computer as pc

Using [pystac_client](https://github.com/stac-utils/pystac-client) we can fetch information about the ASTER L1T collection:

In [465]:
api = API.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [466]:
aster_l1t = api.get_child(id='aster-l1t')

The Planetary Computer ASTER L1T data is for a limited scope in time:

In [467]:
aster_l1t.extent.temporal.to_dict()

{'interval': [['2000-03-04T12:00:00Z', '2006-12-31T12:00:00Z']]}

For this example we'll search 2003 imagery over an area in Redmond, around Microsoft's main campus.

In [468]:
time_of_interest = "2003-01-01/2003-12-31"

In [469]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.27508544921875, 47.54687159892238],
            [-121.96128845214844, 47.54687159892238],
            [-121.96128845214844, 47.745787772920934],
            [-122.27508544921875, 47.745787772920934],
            [-122.27508544921875, 47.54687159892238],
        ]
    ],
}

We'll use this criteria to perform a search against the STAC API:

In [470]:
search = api.search(
    collections=['aster-l1t'], 
    intersects=area_of_interest, 
    datetime=time_of_interest    
)
print(f"{search.matched()} Items found")

48 Items found


We can use the code below to look through the 'browse' thumbnail assets for the Items returned. Notice the usage of the [planetary-computer](https://github.com/microsoft/planetary-computer-sdk-for-python) package's `pc.sign` method - this provides a version of the URL to the image that has a [Shared Access Signature](https://docs.microsoft.com/en-us/azure/storage/common/storage-sas-overview) attached to it, which allows us to use the HREF in tooling that expects a publicly accessible URL.

In [449]:
from IPython.display import Image

In [463]:
from ipywidgets import interact
from IPython.display import display, Image

def browse_images(items, browse_asset='vnir-browse'):
    browse_items = [
        pc.sign_assets(item)
        for item in items
        if browse_asset in item.assets
    ]
    n = len(browse_items)

    def view_image(i=0):
        item = browse_items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item.ext.eo.cloud_cover}")
        display(Image(url=item.assets[browse_asset].href))

    interact(view_image, i=(0,n-1))

In [464]:
browse_images(search.items())

interactive(children=(IntSlider(value=0, description='i', max=43), Output()), _dom_classes=('widget-interact',…